# Censo - parte 2: creación del dataset combinado

Vamos a cargar el dataset del apartado anterior y añadir información adicional de otros ficheros

In [ ]:
# Ensure we are conenected
sc.getConf().get('spark.master')

## Carga de datos

In [ ]:
# Read the data from parquet
censo = spark.read.format('parquet').load('data/censo-2011.parquet')

In [ ]:
# ¿Cuántas particiones tienen los datos?
censo.rdd.getNumPartitions()

In [ ]:
# ¿Qué campos tenemos? 
censo.printSchema()

In [ ]:
# ¿Cuántos campos?
len(censo.columns)

In [ ]:
# ¿Cuántas filas?
nrow = censo.count()
print(nrow)

In [ ]:
# Veamos una muestra
import pandas as pd
with pd.option_context('display.max_columns', 999):
    sample = censo.limit(10).toPandas()

sample

## Municipios

Añadimos el nombre del municipio de la persona encuestada. Para ello leemos la lista de códigos de municipio de un fichero CSV


In [ ]:
cmun = spark.read.format('csv').option("header",True).option("encoding","utf-8").load('data/codes-cmun.csv')

In [ ]:
cmun.printSchema()

In [ ]:
cmun.take(10)

### Join
Ahora unimos ambas tablas, la del censo y la de municipios

In [ ]:
cond = [censo.CMUN == cmun.CMUN, censo.CPRO == cmun.CPRO ]

In [ ]:
# Hacemos el join, y eliminamos las columnas repetidas
censo_codes = censo.join( cmun, cond ).drop( cmun.CPRO ).drop( cmun.CMUN ).drop( cmun.DC )

In [ ]:
censo_codes.count()

In [ ]:
censo_codes.limit(4).toPandas()

### Filas perdidas

Hemos perdido muchas filas al hacer el _join_ ¿Por qué? Veamos qué hemos perdido

In [ ]:
not_cmun = censo.join(cmun, cond, 'left_anti')

In [ ]:
not_cmun.limit(10).toPandas()

## Ejemplo de cálculos sencillos

### Filtrado por municipio

In [ ]:
cac = censo_codes.filter( censo_codes.NOMBRE_MUN == 'Cáceres' )

In [ ]:
cac.count()

In [ ]:
cac.limit(10).toPandas()

### Filtrado de llegadas recientes
Veamos las llegadas recientes a ese municipio

In [ ]:
recientes = cac.filter( cac.ANOM > '2000' )

In [ ]:
recientes.count()

### Cálculo de inmigración
Dentro de las llegadas recientes a ese municipio, cuántos han llegado de fuera de la provincia

In [ ]:
traslados_prov = recientes.filter( recientes.CLPRO != recientes.CPRO )

In [ ]:
traslados_prov.count()

Y veamos de qué provincia han llegado

In [ ]:
A = traslados_prov.groupBy(traslados_prov.CLPRO).count().sort('count', ascending=False)

In [ ]:
A.printSchema()

In [ ]:
A.limit(10).toPandas()

## Inserción de información de provincias

### Lectura de datos

In [ ]:
# Podemos leer datos desde el driver (que luego tendríamos que pasar a Spark)
import csv
with open('data/codes-prov.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    prov = { row[0] : row[1] for row in reader }

In [ ]:
# o directamente a Spark, si los ejecutores tienen acceso a ellos (HDFS o local, según procesda)
cprov = spark.read.format('csv').option("header",True).option('encoding','utf-8').load('data/codes-prov.csv')

In [ ]:
cprov.printSchema()

### join

In [ ]:
A.printSchema()

In [ ]:
cemigracion = cprov.join( A, [A.CLPRO == cprov.CPRO] ).drop('CPRO')

In [ ]:
cemigracion.sort('count', ascending=False).limit(20).toPandas()

## Grabamos a disco

In [ ]:
censo_names = censo_codes.join(cprov, 'CPRO')
censo_names.write.format("parquet").mode("overwrite").save("data/censo-join.parquet")

In [ ]:
censo_names.printSchema()